<a href="https://colab.research.google.com/github/bill7845/colab_repo/blob/master/dacon_%EC%B2%9C%EC%B2%B4%EC%9C%A0%ED%98%95%EB%B6%84%EB%A5%98/%EC%B2%9C%EC%B2%B4%EC%9C%A0%ED%98%95%EB%B6%84%EB%A5%98_NOTE_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import missingno as msno
import scipy as sp

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split,GridSearchCV,KFold,cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score,log_loss
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier,AdaBoostClassifier,VotingClassifier,BaggingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.pipeline import Pipeline

from sklearn.decomposition import PCA
from sklearn.decomposition import KernelPCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_curve
from yellowbrick.classifier import ROCAUC
from sklearn.metrics import classification_report

from xgboost import XGBClassifier
import xgboost as xgb
import lightgbm as lgb
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV

sns.set()

%matplotlib inline
%precision 3

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


'%.3f'

In [0]:
# train = pd.read_csv('C:/Users/KIHyuk/Desktop/dacon_data/Data_천체유형분류/train.csv',index_col=0)
# test = pd.read_csv('C:/Users/KIHyuk/Desktop/dacon_data/Data_천체유형분류/test.csv',index_col=0)
# sample_submission = pd.read_csv('C:/Users/KIHyuk/Desktop/dacon_data/Data_천체유형분류/sample_submission.csv',index_col=0)

train = pd.read_csv('/content/drive/My Drive/Colab Notebooks/dacon_천체유형분류/train.csv',index_col=0)
test = pd.read_csv('/content/drive/My Drive/Colab Notebooks/dacon_천체유형분류/test.csv',index_col=0)
sample_submission = pd.read_csv('/content/drive/My Drive/Colab Notebooks/dacon_천체유형분류/sample_submission.csv',index_col=0)

In [0]:
## data preprocessing

column_number = {}
for i, column in enumerate(sample_submission.columns):
    column_number[column] = i
    
def to_number(x, dic):
    return dic[x]

train['type_num'] = train['type'].apply(lambda x: to_number(x, column_number))

train = train.drop(['fiberID'],axis=1) # del fiberID column
test = test.drop(['fiberID'],axis=1)

# split train/test
# strtify => 클래스 비율 유지
train_x,test_x,train_y,test_y = train_test_split(train.drop(columns=['type', 'type_num'], axis=1),train['type_num'],random_state=2,stratify=train['type_num'])

# for submission
res_train_x = train.drop(columns=['type', 'type_num'], axis=1)
res_train_y = train['type_num']
res_test_x = test

In [0]:
# pca = PCA(n_components=0.95,random_state=2)
# pca_train_x = pca.fit_transform(sc_train_x)
# pca_test_x = pca.transform(sc_test_x) # * transform 

# print('explained variance ratio :', pca.explained_variance_ratio_) # 주성분별 분산 설명 비율 (*설명비율)
# print('선택한 차원(픽셀) 수 :', pca.n_components_) # 95%의 분산 설명을 위해 몇개의 차원을 선택했는가

explained variance ratio : [0.383 0.14  0.079 0.069 0.058 0.049 0.045 0.043 0.033 0.029 0.026]
선택한 차원(픽셀) 수 : 11


In [0]:
data = [train_x,train_y,test_x,test_y]
model_info_lr = ['logistic',LogisticRegression()]
model_info_rf = ['rf',RandomForestClassifier()]
model_info_lgb = ['lgb',lgb.LGBMClassifier()]

grid_params_lr = [{'logistic__penalty': ['l1','l2'],
                   'logistic__C': [0.001,0.01,1,10,100],
                   'logistic__solver': ['lbfgs'],
                   'logistic__random_state':[2],
                   'logistic__multi_class' : ['multinomial']
                  }] 


grid_params_rf = [{'rf__n_estimators' : [100,200,300],
                   'rf__max_depth' : [5,15,25],
                   'rf__min_samples_split' : [1,5,15],
                   'rf__min_samples_leaf' : [1,5,10],
                   'rf__n_jobs' : [-1],
                   'rf__random_state' : [2]
                   }]

grid_params_lgb = [{'lgb__num_leaves' : [500,600],
                    'lgb__n_estimators' : [100,300],
                    'lgb__reg_alpha' : [0.1,0.5],
                    'lgb__min_data_in_leaf' : [30,50],
                    'lgb__objective' : ['softmax'],
                    'lgb__learning_rate' : [0.03,0.01],
                    'lgb__metric' : ['softmax'],
                    'lgb__random_state' : [2],
                    'lgb__max_depth' : [12,-1],
                    'lgb__n_jobs' : [-1]
                    }]           

In [0]:
lgb.LGBMClassifier()

In [0]:
n_estimators=100

In [0]:
# 왜 파이프라인을 통해 교차검증을 진행하는게 중요한가
# https://woolulu.tistory.com/80?category=783167

target_name = list(train['type'].unique())
def modeling(data, model_info, params, target_name): # target_name 고정시키기
    
    train_x,train_y = data[0],data[1]
    test_x,test_y = data[2],data[3]
    
    scaler = ('scaler',StandardScaler())
    model = (model_info[0],model_info[1])
    pipe = Pipeline([scaler,model])
    
    grid = GridSearchCV(pipe, param_grid=params, cv=2,n_jobs=-1,refit=True) # refit => True
    grid.fit(train_x,train_y)
    
    best_param = grid.best_params_
    print("####################################################")
    print("best params:{}".format(grid.best_params_))
    print("####################################################")
    print("\n\n")
    
    predict = grid.predict(test_x) # refit=True => best params로 예측 
    print(classification_report(test_y,predict,target_names=target_name))
    print("\n\n")
    
    predict_proba = grid.predict_proba(test_x)
    print("####################################################")
    print("logloss : ",log_loss(test_y,predict_proba))
    print("####################################################")
    print("\n\n")
    
    result_list = [best_param,predict,predict_proba]
    
    return result_list

In [0]:
modeling(data,model_info_lr,grid_params_lr,target_name)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


####################################################
best params:{'logistic__C': 100, 'logistic__multi_class': 'multinomial', 'logistic__penalty': 'l2', 'logistic__random_state': 2, 'logistic__solver': 'lbfgs'}
####################################################



                     precision    recall  f1-score   support

                QSO       0.00      0.00      0.00       540
     STAR_RED_DWARF       0.77      0.94      0.85      1627
   SERENDIPITY_BLUE       0.00      0.00      0.00       125
           STAR_BHB       0.41      0.09      0.15       640
      STAR_CATY_VAR       0.61      0.57      0.59      3655
SERENDIPITY_DISTANT       0.21      0.13      0.16      3375
             GALAXY       0.87      0.98      0.92      9337
   SPECTROPHOTO_STD       0.00      0.00      0.00      1163
         REDDEN_STD       0.60      0.81      0.69     12420
            ROSAT_D       0.00      0.00      0.00        32
   STAR_WHITE_DWARF       0.75      0.98      0.85      3438


[{'logistic__C': 100,
  'logistic__multi_class': 'multinomial',
  'logistic__penalty': 'l2',
  'logistic__random_state': 2,
  'logistic__solver': 'lbfgs'},
 array([15,  4,  8, ..., 10, 10,  8]),
 array([[4.341e-03, 3.783e-02, 1.553e-04, ..., 1.660e-03, 2.917e-11,
         5.969e-03],
        [2.508e-02, 3.201e-02, 2.534e-03, ..., 7.202e-03, 1.646e-10,
         1.078e-01],
        [3.101e-02, 6.170e-02, 1.336e-02, ..., 2.262e-02, 3.576e-10,
         4.175e-02],
        ...,
        [2.016e-02, 1.317e-01, 2.235e-02, ..., 3.568e-02, 3.824e-10,
         1.632e-02],
        [1.438e-03, 2.067e-02, 7.034e-03, ..., 7.589e-03, 5.128e-11,
         3.475e-04],
        [7.412e-03, 5.651e-03, 2.003e-02, ..., 8.842e-03, 1.311e-10,
         1.643e-02]])]

In [0]:
modeling(data,model_info_rf,grid_params_rf,target_name)

/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


####################################################
best params:{'rf__max_depth': 25, 'rf__min_samples_leaf': 1, 'rf__min_samples_split': 5, 'rf__n_estimators': 300, 'rf__n_jobs': -1, 'rf__random_state': 2}
####################################################



                     precision    recall  f1-score   support

                QSO       0.46      0.42      0.44       540
     STAR_RED_DWARF       0.96      0.97      0.96      1627
   SERENDIPITY_BLUE       0.63      0.38      0.47       125
           STAR_BHB       0.80      0.83      0.82       640
      STAR_CATY_VAR       0.90      0.98      0.93      3655
SERENDIPITY_DISTANT       0.85      0.89      0.87      3375
             GALAXY       0.97      0.98      0.97      9337
   SPECTROPHOTO_STD       0.54      0.44      0.48      1163
         REDDEN_STD       0.89      0.87      0.88     12420
            ROSAT_D       0.82      0.97      0.89        32
   STAR_WHITE_DWARF       0.97      0.98      0.97      3438
   

[{'rf__max_depth': 25,
  'rf__min_samples_leaf': 1,
  'rf__min_samples_split': 5,
  'rf__n_estimators': 300,
  'rf__n_jobs': -1,
  'rf__random_state': 2},
 array([15,  4, 14, ..., 16, 10, 13]),
 array([[0.000e+00, 0.000e+00, 0.000e+00, ..., 0.000e+00, 0.000e+00,
         0.000e+00],
        [3.512e-04, 0.000e+00, 0.000e+00, ..., 0.000e+00, 0.000e+00,
         0.000e+00],
        [7.898e-02, 0.000e+00, 0.000e+00, ..., 0.000e+00, 0.000e+00,
         0.000e+00],
        ...,
        [0.000e+00, 0.000e+00, 0.000e+00, ..., 6.870e-01, 0.000e+00,
         0.000e+00],
        [0.000e+00, 0.000e+00, 6.685e-07, ..., 0.000e+00, 0.000e+00,
         0.000e+00],
        [0.000e+00, 0.000e+00, 0.000e+00, ..., 0.000e+00, 4.990e-06,
         0.000e+00]])]

In [0]:
modeling(data,model_info_lgb,grid_params_lgb,target_name)

/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


####################################################
best params:{'lgb__learning_rate': 0.03, 'lgb__max_depth': 12, 'lgb__metric': 'softmax', 'lgb__min_data_in_leaf': 30, 'lgb__n_estimators': 300, 'lgb__n_jobs': -1, 'lgb__num_leaves': 500, 'lgb__objective': 'softmax', 'lgb__random_state': 2, 'lgb__reg_alpha': 0.1}
####################################################



                     precision    recall  f1-score   support

                QSO       0.46      0.42      0.44       540
     STAR_RED_DWARF       0.96      0.96      0.96      1627
   SERENDIPITY_BLUE       0.61      0.46      0.52       125
           STAR_BHB       0.80      0.82      0.81       640
      STAR_CATY_VAR       0.89      0.97      0.93      3655
SERENDIPITY_DISTANT       0.85      0.88      0.86      3375
             GALAXY       0.97      0.98      0.97      9337
   SPECTROPHOTO_STD       0.53      0.47      0.50      1163
         REDDEN_STD       0.89      0.86      0.88     12420
            ROSAT

[{'lgb__learning_rate': 0.030,
  'lgb__max_depth': 12,
  'lgb__metric': 'softmax',
  'lgb__min_data_in_leaf': 30,
  'lgb__n_estimators': 300,
  'lgb__n_jobs': -1,
  'lgb__num_leaves': 500,
  'lgb__objective': 'softmax',
  'lgb__random_state': 2,
  'lgb__reg_alpha': 0.100},
 array([15,  4, 14, ..., 16, 10, 13]),
 array([[1.185e-03, 2.265e-04, 1.757e-05, ..., 4.030e-05, 8.202e-06,
         8.120e-04],
        [1.480e-03, 2.836e-04, 1.446e-05, ..., 3.316e-05, 4.948e-06,
         1.469e-03],
        [8.613e-02, 1.468e-03, 1.589e-04, ..., 8.806e-05, 8.467e-06,
         1.837e-03],
        ...,
        [7.766e-05, 1.679e-04, 1.242e-04, ..., 8.364e-01, 4.349e-06,
         5.129e-04],
        [9.146e-06, 2.565e-05, 1.291e-05, ..., 1.155e-05, 5.748e-06,
         8.190e-05],
        [3.143e-05, 8.817e-05, 8.353e-06, ..., 1.619e-05, 2.007e-06,
         2.824e-04]])]